In [ ]:
#General
import re
import sys
import pprint
import seaborn as sb
import numpy   as np
import pandas  as pd
import random  as rnd

from os.path import isfile
from copy    import deepcopy
from time    import time
from tqdm    import tqdm
from getdist import plots

from scipy.interpolate import interp1d
from scipy.stats       import multivariate_normal
from itertools         import repeat

from getdist import plots,loadMCSamples,MCSamples


from analysis.chains_analysis import Analyzer
from theory_code.distance_theory import TheoryCalcs

#Plotting
import matplotlib
import matplotlib.pyplot as plt


from matplotlib import rc

rc('text', usetex=True)
rc('font', family='serif')
matplotlib.rcParams.update({'font.size': 18})

red    = '#8e001c'
yellow = '#ffb302'

sidelegend = {'bbox_to_anchor': (1.04,0.5), 
              'loc': "center left",
              'frameon': False}
bottomlegend = {'bbox_to_anchor': (0.35,-0.2), 
                'loc': "center left",
                'frameon': False,
                'ncols': 3}

In [ ]:
fiducial = {'$\Lambda$CDM':{'H0': 68.53,
                            'omegam': 0.304,
                            'ombh2': 0.02218,
                            'epsilon0_EM': 0.0,
                            'epsilon0_GW': 0.0,
                            'omk': 0.,
                            'mnu': 0.06,
                            'nnu': 3.,
                            'MB': -19.2435},
            '$\epsilon_{EM}$ breaking': {'H0': 68.53,
                            'omegam': 0.304,
                            'ombh2': 0.02218,
                            'epsilon0_EM': 0.1,
                            'epsilon0_GW': 0.0,
                            'omk': 0.,
                            'mnu': 0.06,
                            'nnu': 3.,
                            'MB': -19.2435},
            '$\epsilon$ breaking': {'H0': 68.53,
                            'omegam': 0.304,
                            'ombh2': 0.02218,
                            'epsilon0_EM': 0.1,
                            'epsilon0_GW': 0.1,
                            'omk': 0.,
                            'mnu': 0.06,
                            'nnu': 3.,
                            'MB': -19.2435},
            '$\epsilon_0 \, \Lambda$CDM':{'H0': 68.53,
                            'omegam': 0.304,
                            'ombh2': 0.02218,
                            'epsilon0_EM': 0.0,
                            'epsilon0_GW': 0.0,
                            'omk': 0.,
                            'mnu': 0.06,
                            'nnu': 3.,
                            'MB': -19.2435},
            '$\epsilon_0$':{'H0': 68.53,
                            'omegam': 0.304,
                            'ombh2': 0.02218,
                            'epsilon0_EM': 0.1,
                            'epsilon0_GW': 0.1,
                            'omk': 0.,
                            'mnu': 0.06,
                            'nnu': 3.,
                            'MB': -19.2435},
            }


theory_settings = {'zmin': 0.001,
                   'zmax': 5.,
                   'Nz': 1000,
                   'zdrag': 1060}

zplot = np.linspace(0.01,3,100)
use_estimator = 'Mean'

analysis_settings = {'Metropolis-Hastings': {'ignore_rows': 0.3},
                     'Nautilus': {}}


BAO_data = './mock_data/data/SKAOmock'
SN_data  = './mock_data/data/SNmock'
SN_data_breaking  = './mock_data/data/SNmock_breaking'
GW_data  = './mock_data/data/GWmock'
GW_data_breaking  = './mock_data/data/GWmock_breaking'


In [ ]:
plot_epsilonEMGW = {'$\Lambda$CDM': {'path': './DDR_chains/chains_LCDM/BAO+SN+GW_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                       'color': 'orange',
                                       'filled': True},
                    '$\epsilon_{EM}$ breaking': {'path': './DDR_chains/chains_epsilonEM/BAO+SN+GW_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                       'color': 'hotpink',
                                       'filled': True},
                    '$\epsilon$ breaking': {'path': './DDR_chains/chains_epsilonEMGW/BAO+SN+GW_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                       'color': 'cornflowerblue',
                                       'filled': True},
    }

In [ ]:
stats = Analyzer(analysis_settings,fiducial)

In [ ]:
analyzed_results_plotEMGW = {name: stats.analyze_chain(name,resdict) for name,resdict in plot_epsilonEMGW.items()}

# $\epsilon_{\rm EM}$ vs $\epsilon_{\rm GW}$ plot

In [ ]:
plot_pars = ['epsilon0_EM', 'epsilon0_GW']

### SN + GW + BAO

In [ ]:
import matplotlib.pyplot as plt
from getdist import plots

g = plots.get_subplot_plotter(subplot_size=1, width_inch=10, scaling=False)

g.settings.figure_legend_frame = False
g.settings.axes_fontsize = 24
g.settings.axes_labelsize = 24
g.settings.legend_fontsize = 12
g.settings.axis_marker_color = 'black'
g.settings.axis_marker_ls = '--'
g.settings.axis_marker_lw = 1
g.settings.axis_tick_x_rotation = 45

g.plot_2d([result['MCsamples'] for result in analyzed_results_plotEMGW.values()], 
            plot_pars, 
            plot_pars, 
            filled=[result['filled'] for result in analyzed_results_plotEMGW.values()], 
            colors=[result['color'] for result in analyzed_results_plotEMGW.values()], 
            contour_lws=2,
        )


g.fig.align_ylabels()
g.fig.align_xlabels()

ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()
min_limit = max(xlim[0], ylim[0])
max_limit = min(xlim[1], ylim[1])
plt.plot([min_limit, max_limit], [min_limit, max_limit], '--k', linewidth=1)

colors = [result['color'] for result in analyzed_results_plotEMGW.values()]
labels = ['$\Lambda$CDM', 'EM breaking', 'full breaking']
handles = [plt.Line2D([0], [0], color=color, lw=2, label=label) for color, label in zip(colors, labels)]
plt.legend(handles=handles, loc='upper left', fontsize=26, frameon=False)

plt.savefig('epsilonEMGW.png', bbox_inches='tight', dpi=300)

plt.show()


In [ ]:
FoM_plot = { '$\Lambda$CDM': {
                            
                            r'SN+GW': {'path': './DDR_chains/chains_LCDM/SN+GW_MH',
                                                 'sampler': 'MH',
                                                 'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                                 'color': 'darkorange',
                                                 'Nchains': 4,
                                                 'filled': True},
                            r'BAO+SN+GW': {'path': './DDR_chains/chains_LCDM/BAO+SN+GW_MH',
                                                 'sampler': 'MH',
                                                 'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                                 'color': 'cornflowerblue',
                                                 'Nchains': 4,
                                                 'filled': True},
                         
                             },
        '$\epsilon_{EM}$ breaking': {
                            r'SN+GW': {'path': './DDR_chains/chains_epsilonEM/SN+GW_MH',
                                                 'sampler': 'MH',
                                                 'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                                 'color': 'darkorange',
                                                 'Nchains': 4,
                                                 'filled': True},
                            r'BAO+SN+GW': {'path': './DDR_chains/chains_epsilonEM/BAO+SN+GW_MH',
                                                 'sampler': 'MH',
                                                 'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                                 'color': 'cornflowerblue',
                                                 'Nchains': 4,
                                                 'filled': True},
                            
                            },
        '$\epsilon$ breaking': {
           
                            r'SN+GW': {'path': './DDR_chains/chains_epsilonEM/SN+GW_MH',
                                                 'sampler': 'MH',
                                                 'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                                 'color': 'darkorange',
                                                 'Nchains': 4,
                                                 'filled': True},
                             r'BAO+SN+GW': {'path': './DDR_chains/chains_epsilonEMGW/BAO+SN+GW_MH',
                                                 'sampler': 'MH',
                                                 'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                                 'color': 'cornflowerblue',
                                                 'Nchains': 4,
                                                 'filled': True},
                            },
        }   
                               

In [ ]:
analyzed_results_FoM = {model: {name: stats.analyze_chain(name,resdict) for name,resdict in model_dict.items()} 
                    for model,model_dict in FoM_plot.items()}

In [ ]:
FoM_results = {}
sigma_results = {}

for model, cases in analyzed_results_FoM.items():
    FoM_results[model] = {}
    sigma_results[model] = {}
    for case, result in cases.items():
        samples = result['MCsamples']  
        sub_cov_matrix = samples.cov(pars=plot_pars[:2])  
        det_cov = np.linalg.det(sub_cov_matrix)

        
        if det_cov > 0: 
            FoM = 1 / np.sqrt(det_cov)
        else:
            FoM = np.nan 
        FoM_results[model][case] = FoM
        sigma_results[model][case] = np.sqrt(np.diag(sub_cov_matrix))  # [σ1, σ2]



FoM_df = pd.DataFrame(FoM_results).T  


plt.figure(figsize=(6, 6))

# Define x positions for different cases
cases = FoM_df.columns.tolist()
x_positions = np.arange(len(cases))
color = [ 'orange', 'hotpink', 'cornflowerblue']
labels = ['$\Lambda$CDM', '$\epsilon_{EM}$ breaking', '$\epsilon_{EM}$, $\epsilon_{GW}$ breaking']
for i, (model, values) in enumerate(FoM_df.iterrows()):
    plt.plot(x_positions, values, marker='o', linestyle='-', markersize=8, label=labels[i], color=color[i],  alpha=0.7)

plt.xlabel("Case")
plt.ylabel("Figure of Merit (FoM)")
plt.xticks(x_positions, cases, rotation=45, ha='right')
plt.legend()


# Show plot
plt.show()
linestyle = ['-', '--', ':']
lwd = [4, 3, 2]
alpha = [0.5, 0.7, 1]


sigmas_param1 = pd.DataFrame({model: {case: sigmas[0] for case, sigmas in cases.items()}
                              for model, cases in sigma_results.items()})
sigmas_param2 = pd.DataFrame({model: {case: sigmas[1] for case, sigmas in cases.items()}
                              for model, cases in sigma_results.items()})

for param_idx, (df, pname) in enumerate(zip([sigmas_param1, sigmas_param2], plot_pars[:2])):
    plt.figure(figsize=(6, 6))
    for i, model in enumerate(df.columns):
        values = df[model]
        plt.plot(x_positions, values, marker='s', linestyle=linestyle[i], markersize=8, label=f"{labels[i]} ", color=colors[i], alpha=alpha[i], zorder=i)
    plt.yscale('log')
    plt.ylabel(r"$\sigma$")
    plt.xticks(x_positions, cases, rotation=45, ha='right')
    plt.legend( frameon =False)
    plt.tight_layout()
    plt.show()


In [ ]:
FoM_plot_omegam_epsilon0 = { '$\epsilon_0 \, \Lambda$CDM': {
                        r'SN+GW': {'path': './DDR_chains/chains_epsilon0_LCDM/SN+GW_MH',
                                                 'sampler': 'MH',
                                                 'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                                 'color': 'forestgreen',
                                                 'Nchains': 4,
                                       'alpha': 0.5,
                                                 'filled': True},
                        r'BAO+GW': {'path': './DDR_chains/chains_epsilon0_LCDM/BAO+GW_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                      'alpha': 0.5,
                                       'color': 'orange',
                                       'filled': True},
                            
                            r'BAO+SN': {'path': './DDR_chains/chains_epsilon0_LCDM/BAO+SN_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                        'alpha': 0.5,
                                       'color': 'hotpink',
                                       'filled': True},
                            
                            r'BAO+SN+GW': {'path': './DDR_chains/chains_epsilon0_LCDM/BAO+SN+GW_MH',
                                                 'sampler': 'MH',
                                                 'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                                 'color': 'cornflowerblue',
                                                 'Nchains': 4,
                                           'alpha': 1,
                                                 'filled': True},
                            },
            '$\epsilon_0$': {
                            r'SN+GW': {'path': './DDR_chains/chains_epsilon0/SN+GW_MH',
                                                 'sampler': 'MH',
                                                 'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                                 'color': 'forestgreen',
                                                 'Nchains': 4,
                                                 'alpha': 0.5,
                                                 'filled': True},
                            r'BAO+GW': {'path': './DDR_chains/chains_epsilon0/BAO+GW_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                       'color': 'orange',
                                       'alpha': 0.5,
                                       'filled': True},
                            r'BAO+SN': {'path': './DDR_chains/chains_epsilon0/BAO+SN_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                        'alpha': 0.5,
                                       'color': 'hotpink',
                                       'filled': True},
                            r'BAO+SN+GW': {'path': './DDR_chains/chains_epsilon0/BAO+SN+GW_MH',
                                                 'sampler': 'MH',
                                                 'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                                 'color': 'cornflowerblue',
                                                 'Nchains': 4,
                                                 'alpha': 1,
                                                 'filled': True},
                            },
        }   
                               

In [ ]:
plot_pars = ['omegam', 'epsilon0_EM']

In [ ]:
FoM_plot_omegam_epsilon0 = {model: {name: stats.analyze_chain(name,resdict) for name,resdict in model_dict.items()} 
                    for model,model_dict in FoM_plot_omegam_epsilon0.items()}

In [ ]:
FoM_results = {}
for model, cases in FoM_plot_omegam_epsilon0.items():
    FoM_results[model] = {}
    for case, result in cases.items():
        samples = result['MCsamples']  

        sub_cov_matrix = samples.cov(pars=plot_pars[:2])  
        det_cov = np.linalg.det(sub_cov_matrix)

        
        if det_cov > 0: 
            FoM = 1 / np.sqrt(det_cov)
        else:
            FoM = np.nan 
        FoM_results[model][case] = FoM



FoM_df = pd.DataFrame(FoM_results).T  


plt.figure(figsize=(6, 6))

# Define x positions for different cases
cases = FoM_df.columns.tolist()
x_positions = np.arange(len(cases))
color = ['orange', 'hotpink']
labels = ['$\Lambda$CDM', 'full breaking']
for i, (model, values) in enumerate(FoM_df.iterrows()):
    plt.plot(x_positions, values, marker='o', linestyle='-', markersize=8, label=labels[i], color=color[i],  alpha=0.7)

plt.xlabel("Case")
plt.ylabel("Figure of Merit (FoM)")
plt.xticks(x_positions, cases, rotation=45, ha='right')
plt.legend()


# Show plot
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Existing FoM calculation
FoM_results = {}
for model, cases in FoM_plot_omegam_epsilon0.items():
    FoM_results[model] = {}
    for case, result in cases.items():
        samples = result['MCsamples']
        sub_cov_matrix = samples.cov(pars=plot_pars[:2])
        det_cov = np.linalg.det(sub_cov_matrix)
        FoM = 1 / np.sqrt(det_cov) if det_cov > 0 else np.nan
        FoM_results[model][case] = FoM

FoM_df = pd.DataFrame(FoM_results).T
linestyle = ['-', '--']
lwd = [3, 2]

plt.figure(figsize=(6, 6))
cases = FoM_df.columns.tolist()
x_positions = np.arange(len(cases))
colors = ['cornflowerblue', 'hotpink']
labels = ['$\Lambda$CDM', 'full breaking']

for i, (model, values) in enumerate(FoM_df.iterrows()):
    plt.plot(x_positions, values, marker='o', linestyle=linestyle[i], markersize=8, label=labels[i], color=colors[i], alpha=0.7)

plt.yscale('log')
plt.ylabel("Figure of Merit (FoM)")
plt.xticks(x_positions, cases, rotation=45, ha='right')
plt.legend(frameon=False)
plt.tight_layout()
plt.show()

# Sigma (standard deviation) calculation
sigma_results = {model: {} for model in FoM_plot_omegam_epsilon0}
for model, cases in FoM_plot_omegam_epsilon0.items():
    for case, result in cases.items():
        samples = result['MCsamples']
        cov_matrix = samples.cov(pars=plot_pars[:2])
        sigma = np.sqrt(np.diag(cov_matrix))
        sigma_results[model][case] = sigma

# Convert to DataFrame per parameter
sigmas_param1 = pd.DataFrame({model: {case: sigma[0] for case, sigma in cases.items()} for model, cases in sigma_results.items()})
sigmas_param2 = pd.DataFrame({model: {case: sigma[1] for case, sigma in cases.items()} for model, cases in sigma_results.items()})

# Plot sigmas for both parameters
for param_idx, (df, pname) in enumerate(zip([sigmas_param1, sigmas_param2], plot_pars[:2])):
    plt.figure(figsize=(10, 10))
    for i, model in enumerate(df.columns):
        values = df[model]
        plt.plot(x_positions, values, marker='s', linestyle=linestyle[i],linewidth=3, markersize=8, label=f"{labels[i]} ", color=colors[i], alpha=0.7)
    #plt.yscale('log')
    ax = plt.gca()
    ax.set_ylabel(r"$\sigma_{\epsilon_0}$", fontsize=36, labelpad=20)  # Moved left with labelpad=20
    plt.xticks(x_positions, cases, rotation=45, ha='right', fontsize=32)
    plt.yticks(fontsize=32)
    plt.legend(frameon=False, fontsize=32)
    plt.tight_layout()
    
    plt.savefig(f'sigma_{pname}.png', bbox_inches='tight', dpi=300)
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
from getdist import plots


for model, cases in FoM_plot_omegam_epsilon0.items():
    bao_keys = {key: value for key, value in cases.items() if 'BAO' in key}
    
    if bao_keys:
        g = plots.get_subplot_plotter(subplot_size=1, width_inch=10, scaling=False)

        g.settings.figure_legend_frame = False
        g.settings.axes_fontsize = 20
        g.settings.axes_labelsize = 20
        g.settings.legend_fontsize = 20
        g.settings.axis_marker_color = 'black'
        g.settings.axis_marker_ls = '--'
        g.settings.axis_marker_lw = 1
        g.settings.axis_tick_x_rotation = 45
        print(plot_pars)
        g.plot_2d([result['MCsamples'] for result in bao_keys.values()], 
                    plot_pars, 
                    plot_pars, 
                    filled=[result['filled'] for result in bao_keys.values()], 
                    colors=[result['color'] for result in bao_keys.values()], 
                    alphas=[result['alpha'] for result in bao_keys.values()], 
                    contour_lws=2,
                )
        for ax in g.subplots[:, 0]: 
            ax.set_ylabel('') 
          
        
        g.fig.supylabel('$\epsilon_0$')

        g.add_legend([result for result in bao_keys.keys()])
        g.fig.align_ylabels()
        g.fig.align_xlabels()



plt.show()




In [ ]:
plot_omegam_epsilon0 = { r'BAO+GW': {'path': './DDR_chains/chains_epsilon0EM/BAO+GW_MH', #_LCDM/BAO+GW_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                       'color': 'hotpink',
                                       'filled': True},
                           
                        
                         
                            r'BAO+SN': {'path': './DDR_chains/chains_epsilon0EM/BAO+SN_MH',#/BAO+SN_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                       'color': 'cornflowerblue',
                                       'filled': True},
                            }
        

In [ ]:
analyzed_results_plot_omegameps0 = {name: stats.analyze_chain(name,resdict) for name,resdict in plot_omegam_epsilon0.items()}

In [ ]:
import matplotlib.pyplot as plt
from getdist import plots

g = plots.get_subplot_plotter(subplot_size=6, width_inch=8, scaling=False)

g.settings.figure_legend_frame = False
g.settings.axes_fontsize = 32
g.settings.axes_labelsize = 32
g.settings.legend_fontsize = 32
g.settings.axis_marker_color = 'black'
g.settings.axis_marker_ls = '--'
g.settings.axis_marker_lw = 1
g.settings.axis_tick_x_rotation = 45

g.plot_2d([result['MCsamples'] for result in analyzed_results_plot_omegameps0.values()], 
            plot_pars, 
            plot_pars, 
            filled=[result['filled'] for result in analyzed_results_plot_omegameps0.values()], 
            colors=[result['color'] for result in analyzed_results_plot_omegameps0.values()], 
            contour_lws=2,
        )
for ax in g.subplots[:, 0]: 
    ax.set_ylabel('') 
          
       
g.fig.supylabel('$\epsilon_0$', fontsize=32)
legend = g.add_legend(
    [result for result in analyzed_results_plot_omegameps0.keys()],
    legend_loc='center right',
    frameon=False
)
legend.get_frame().set_linewidth(0)
legend.get_frame().set_edgecolor('none')

g.fig.align_ylabels()
g.fig.align_xlabels()

plt.savefig('epsilon0_omegam.png', bbox_inches='tight', dpi=300)

plt.show()


In [ ]:
plot_omegam_epsilonEM = {

                           '$\epsilon_{EM}$ breaking': {'path': './DDR_chains/chains_epsilonEM/BAO+SN+GW_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                       'color': 'hotpink',
                                       'filled': True},
                    '$\epsilon_{EM}\,,\epsilon_{GW}$ breaking': {'path': './DDR_chains/chains_epsilonEMGW/BAO+SN+GW_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                       'color': 'cornflowerblue',
                                       'filled': True},
    
        }   
                               

In [ ]:
analyzed_results_plot_omegamepssEM = {name: stats.analyze_chain(name,resdict) for name,resdict in plot_omegam_epsilonEM.items()}


In [ ]:
import matplotlib.pyplot as plt
from getdist import plots

g = plots.get_subplot_plotter(subplot_size=1, width_inch=10, scaling=False)

g.settings.figure_legend_frame = False
g.settings.axes_fontsize = 20
g.settings.axes_labelsize = 20
g.settings.legend_fontsize = 22
g.settings.axis_marker_color = 'black'
g.settings.axis_marker_ls = '--'
g.settings.axis_marker_lw = 1
g.settings.axis_tick_x_rotation = 45

g.plot_2d([result['MCsamples'] for result in analyzed_results_plot_omegamepssEM.values()], 
            plot_pars, 
            plot_pars, 
            filled=[result['filled'] for result in analyzed_results_plot_omegamepssEM.values()], 
            colors=[result['color'] for result in analyzed_results_plot_omegamepssEM.values()], 
            contour_lws=2,
        )


g.fig.align_ylabels()
g.fig.align_xlabels()

ax = plt.gca()
g.settings.figure_legend_frame = False
legend = g.add_legend([result for result in analyzed_results_plot_omegamepssEM.keys()])
legend.get_frame().set_linewidth(0)
legend.get_frame().set_edgecolor('none')

plt.savefig('epsilonEMGW_omegam.png', bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from getdist import plots


for model, cases in FoM_plot_omegam_epsilon0.items():
    bao_keys = {key: value for key, value in cases.items() if 'BAO' in key}
    
    if bao_keys:
        g = plots.get_subplot_plotter(subplot_size=1, width_inch=10, scaling=False)

        g.settings.figure_legend_frame = False
        g.settings.axes_fontsize = 20
        g.settings.axes_labelsize = 20
        g.settings.legend_fontsize = 20
        g.settings.axis_marker_color = 'black'
        g.settings.axis_marker_ls = '--'
        g.settings.axis_marker_lw = 1
        g.settings.axis_tick_x_rotation = 45
        g.plot_2d([result['MCsamples'] for result in bao_keys.values()], 
                    plot_pars, 
                    plot_pars, 
                    filled=[result['filled'] for result in bao_keys.values()], 
                    colors=[result['color'] for result in bao_keys.values()], 
                    contour_lws=2,

                )
        for ax in g.subplots[:, 0]: 
            ax.set_ylabel('') 
          
        
        g.fig.supylabel('$\epsilon_0$')

        g.add_legend([result for result in bao_keys.keys()])
        g.fig.align_ylabels()
        g.fig.align_xlabels()



plt.show()




In [ ]:
pade_vs_standard = { r'$\epsilon_0 $': { 'BAO+SN+GW': {'path': './DDR_chains/chains_epsilon0/BAO+SN+GW_MH',
                                        'sampler': 'MH',
                                        'DDR_model': {'epsilon_model': 'polynomial',
                                                      'pade': False,},
                                        'Nchains': 4,
                                        'plot_pars': ['omegam', 'epsilon0_EM' ],
                                        'color': 'cornflowerblue',
                                        'filled': True},
                             'BAO+SN+GW Pade': {'path': './pade_chains/chains_pade_epsilon0/BAO+SN+GW_MH', 
                                        'sampler': 'MH',
                                        'DDR_model': {'epsilon_model': 'polynomial',
                                                      'pade': True,},
                                        'Nchains': 4,
                                        'plot_pars': ['omegam', 'epsilon0_EM' ],
                                        'color': 'orange',
                                        'filled': True},
                             },
                    r'$\epsilon_{EM}$': { 'BAO+SN+GW': {'path': './DDR_chains/chains_epsilonEM/BAO+SN+GW_MH',
                                        'sampler': 'MH',
                                        'DDR_model': {'epsilon_model': 'polynomial',
                                                      'pade': False,},
                                        'Nchains': 4,
                                        'plot_pars': ['omegam', 'epsilon0_EM'],
                                        'color': 'cornflowerblue',
                                        'filled': True},
                                'BAO+SN+GW Pade': {'path': './pade_chains/chains_pade_epsilonEM/BAO+SN+GW_MH',
                                        'sampler': 'MH',
                                        'DDR_model': {'epsilon_model': 'polynomial',
                                                      'pade': True,},
                                        'Nchains': 4,
                                        'plot_pars': [ 'omegam', 'epsilon0_EM'],
                                        'color': 'orange',
                                        'filled': True},
                                },
                    r'$\epsilon_{EM}\,,\epsilon_{GW}$': { 'BAO+SN+GW': {'path': './DDR_chains/chains_epsilonEMGW/BAO+SN+GW_MH',
                                        'sampler': 'MH',
                                        'DDR_model': {'epsilon_model': 'polynomial',
                                                      'pade': False,},
                                        'Nchains': 4,
                                        'color': 'cornflowerblue',
                                        'plot_pars': [ 'epsilon0_GW', 'epsilon0_EM'],
                                        'filled': True},
                                'BAO+SN+GW Pade': {'path': './pade_chains/chains_pade_epsilonEMGW/BAO+SN+GW_MH',
                                        'sampler': 'MH',
                                        'DDR_model': {'epsilon_model': 'polynomial',
                                                      'pade': True,},
                                        'Nchains': 4,
                                        'plot_pars': ['epsilon0_GW','epsilon0_EM'],
                                        'color': 'orange',
                                        'filled': True},
                                },
          }

In [ ]:
analyzed_results_pade = {model: {name: stats.analyze_chain(name,resdict) for name,resdict in model_dict.items()} 
                         for model,model_dict in pade_vs_standard.items()}



In [ ]:
plot_pars = {}
for model in pade_vs_standard.keys():
    plot_pars[model] = {}
    for case in pade_vs_standard[model].keys():
        plot_pars[model][case] = pade_vs_standard[model][case]['plot_pars']

In [ ]:
for model, cases in analyzed_results_pade.items():
    g = plots.get_subplot_plotter(subplot_size=1, width_inch=10, scaling=False)
    g.settings.figure_legend_frame = False
    g.settings.axes_fontsize = 20
    g.settings.axes_labelsize = 20
    g.settings.legend_fontsize = 20
    g.settings.axis_marker_color = 'black'
    g.settings.axis_marker_ls = '--'
    g.settings.axis_marker_lw = 1
    g.settings.axis_tick_x_rotation = 45

    legend_labels = []

    for case_name, result in cases.items():
        plot_pars = pade_vs_standard[model][case_name]['plot_pars']
        g.plot_2d(
            result['MCsamples'],
            plot_pars[0],
            plot_pars[1],
            filled=result['filled'],
            colors=[result['color']],
            contour_lws=2,
        )
        if plot_pars[0] == 'epsilon0_GW':
                ax = g.get_axes() 
                ax.set_xlim(0.05, 0.12) 
        
        legend_labels.append(case_name)


    g.add_legend(legend_labels)


## Mock Data generated with $\epsilon$ const model and analyzed with the correct and wrong (Padè) model

In [ ]:
const_data_wrong = { r'$\epsilon_0$': { 
                                r'$\epsilon$ constant': {'path': './DDR_chains/chains_epsilon0/BAO+SN+GW_MH',
                                                                                'sampler': 'MH',
                                                                                'DDR_model': {'epsilon_model': 'polynomial',
                                                                                        'pade': False,},
                                                                                'Nchains': 4,
                                                                                'plot_pars': ['omegam', 'epsilon0_EM' ],
                                                                                'color': 'hotpink',
                                                                                'filled': True},
                                'Padè': {'path': './chains_wrong/chains_wrong_epsilon0/BAO+SN+GW_MH',
                                                                'sampler': 'MH',
                                                                'DDR_model': {'epsilon_model': 'polynomial',
                                                                        'pade': False,},
                                                                'Nchains': 4,
                                                                'plot_pars': ['omegam', 'epsilon0_EM' ],
                                                                'color': 'cornflowerblue',
                                                                'filled': True},
                                                
                                        },
                r'$\epsilon_{EM}$ breaking':  { 
                                                r'$\epsilon$ constant': {'path': './DDR_chains/chains_epsilonEM/BAO+SN+GW_MH',
                                                                'sampler': 'MH',
                                                                'DDR_model': {'epsilon_model': 'polynomial',
                                                                        'pade': False,},
                                                                'Nchains': 4,
                                                                'plot_pars': ['omegam', 'epsilon0_EM' ],
                                                                'color': 'hotpink',
                                                                'filled': True},
                                                'Padè': {'path': './chains_wrong/chains_wrong_epsilonEM/BAO+SN+GW_MH',
                                                                'sampler': 'MH',
                                                                'DDR_model': {'epsilon_model': 'polynomial',
                                                                        'pade': False,},
                                                                'Nchains': 4,
                                                                'plot_pars': ['omegam', 'epsilon0_EM'],
                                                                'color': 'cornflowerblue',
                                                                'filled': True},
                                },
                    r'$\epsilon$ breaking': {
                                         r'$\epsilon$ constant': {'path': './DDR_chains/chains_epsilonEMGW/BAO+SN+GW_MH',
                                                                'sampler': 'MH',
                                                                'DDR_model': {'epsilon_model': 'polynomial',
                                                                        'pade': False,},
                                                                'Nchains': 4,
                                                                'plot_pars': ['omegam', 'epsilon0_EM' ],
                                                                'color': 'hotpink',
                                                                'filled': True},
                                        'Padè': {'path': './chains_wrong/chains_wrong_epsilonEMGW/BAO+SN+GW_MH',
                                                                'sampler': 'MH',
                                                                'DDR_model': {'epsilon_model': 'polynomial',
                                                                        'pade': False,},
                                                                'Nchains': 4,
                                                                'color': 'cornflowerblue',
                                                                'plot_pars': [ 'epsilon0_GW', 'epsilon0_EM'],
                                                                'filled': True},
                                }
}

In [ ]:
analyzed_results_const_data_wrong = {model: {name: stats.analyze_chain(name,resdict) for name,resdict in model_dict.items()} 
                         for model,model_dict in const_data_wrong.items()}



In [ ]:
plot_pars = [ 'epsilon0_GW', 'epsilon0_EM']
g = plots.get_subplot_plotter(subplot_size=6, width_inch=8, scaling=False)

g.settings.figure_legend_frame = False
g.settings.axes_fontsize = 30
g.settings.axes_labelsize = 30
g.settings.legend_fontsize = 26
g.settings.axis_marker_color = 'black'
g.settings.axis_marker_ls = '--'
g.settings.axis_marker_lw = 1
g.settings.axis_tick_x_rotation = 45


g.plot_2d([result['MCsamples'] for result in analyzed_results_const_data_wrong['$\epsilon$ breaking'].values()], 
            plot_pars, 
            plot_pars, 
            filled=[result['filled'] for result in analyzed_results_const_data_wrong['$\epsilon$ breaking'].values()], 
            colors=[result['color'] for result in analyzed_results_const_data_wrong['$\epsilon$ breaking'].values()], 
            contour_lws=2,
        )


          
ax = plt.gca()
for ax in g.subplots[:, 0]:
    ax.set_ylabel(ax.get_ylabel(), labelpad=15)         
plt.axvline(
    x=fiducial['$\epsilon$ breaking']['epsilon0_GW'], 
    color='black', 
    linestyle='--', 
    linewidth=1, 
)

plt.axhline(
    y=fiducial['$\epsilon$ breaking']['epsilon0_EM'], 
    color='black', 
    linestyle='--', 
    linewidth=1, 
)
legend = g.add_legend(
    [result for result in analyzed_results_const_data_wrong['$\epsilon$ breaking'].keys()],
    legend_loc='lower right',
    frameon=False
)
legend.get_frame().set_linewidth(0)
legend.get_frame().set_edgecolor('none')

g.fig.align_ylabels()
g.fig.align_xlabels()

g.fig.align_ylabels()
g.fig.align_xlabels()

plt.text(0.073, 0.122, "IVS: full breaking", color='black', fontsize=25, ha='left')
plt.savefig('epsilonEMGW_epsilonconst_wrong.png', bbox_inches='tight', dpi=300)

plt.show()


In [ ]:
plot_pars = [ 'omegam', 'epsilon0_EM']
g = plots.get_subplot_plotter(subplot_size=6, width_inch=8, scaling=False)

g.settings.figure_legend_frame = False
g.settings.axes_fontsize = 30
g.settings.axes_labelsize = 30
g.settings.legend_fontsize = 26
g.settings.axis_marker_color = 'black'
g.settings.axis_marker_ls = '--'
g.settings.axis_marker_lw = 1
g.settings.axis_tick_x_rotation = 45


g.plot_2d([result['MCsamples'] for result in analyzed_results_const_data_wrong['$\epsilon_{EM}$ breaking'].values()], 
            plot_pars, 
            plot_pars, 
            filled=[result['filled'] for result in analyzed_results_const_data_wrong['$\epsilon_{EM}$ breaking'].values()], 
            colors=[result['color'] for result in analyzed_results_const_data_wrong['$\epsilon_{EM}$ breaking'].values()], 
            contour_lws=2,
        )

ax = plt.gca()
for ax in g.subplots[:, 0]:
    ax.set_ylabel(ax.get_ylabel(), labelpad=15)  


plt.axvline(
    x=fiducial['$\epsilon_{EM}$ breaking']['omegam'], 
    color='black', 
    linestyle='--', 
    linewidth=1, 
)

plt.axhline(
    y=fiducial['$\epsilon_{EM}$ breaking']['epsilon0_EM'], 
    color='black', 
    linestyle='--', 
    linewidth=1, 
)
legend = g.add_legend(
    [result for result in analyzed_results_const_data_wrong['$\epsilon_{EM}$ breaking'].keys()],
    legend_loc='best',
    frameon=False
)
legend.get_frame().set_linewidth(0)
legend.get_frame().set_edgecolor('none')


g.fig.align_ylabels()
g.fig.align_xlabels()


plt.text(0.305, 0.12, "IVS: EM" "\n" "breaking", color='black', fontsize=25, ha='left')


plt.savefig('epsilonEM_epsilonconst_wrong.png', bbox_inches='tight', dpi=300)
plt.show()




In [ ]:
plot_pars = [ 'omegam', 'epsilon0_EM']
g = plots.get_subplot_plotter(subplot_size=6, width_inch=8, scaling=False)

g.settings.figure_legend_frame = False
g.settings.axes_fontsize = 30
g.settings.axes_labelsize = 30
g.settings.legend_fontsize = 26
g.settings.axis_marker_color = 'black'
g.settings.axis_marker_ls = '--'
g.settings.axis_marker_lw = 1
g.settings.axis_tick_x_rotation = 45


g.plot_2d([result['MCsamples'] for result in analyzed_results_const_data_wrong['$\epsilon_0$'].values()], 
            plot_pars, 
            plot_pars, 
            filled=[result['filled'] for result in analyzed_results_const_data_wrong['$\epsilon_0$'].values()], 
            colors=[result['color'] for result in analyzed_results_const_data_wrong['$\epsilon_0$'].values()], 
            contour_lws=2,
        )
for ax in g.subplots[:, 0]: 
    ax.set_ylabel('') 
          
g.fig.supylabel('$\epsilon_0$', fontsize=32) 
    
plt.axvline(
    x=fiducial['$\epsilon_0$']['omegam'], 
    color='black', 
    linestyle='--', 
    linewidth=1, 
)

plt.axhline(
    y=fiducial['$\epsilon_0$']['epsilon0_EM'], 
    color='black', 
    linestyle='--', 
    linewidth=1, 
)
legend = g.add_legend(
    [result for result in analyzed_results_const_data_wrong['$\epsilon_0$'].keys()],
    legend_loc='best',
    frameon=False
)
legend.get_frame().set_linewidth(0)
legend.get_frame().set_edgecolor('none')

g.fig.align_ylabels()
g.fig.align_xlabels()
plt.text(0.305, 0.119, r"UVS: $\epsilon_0$" "\n" " breaking", color='black', fontsize=25, ha='left')


plt.savefig('epsilon0_epsilonconst_wrong.png', bbox_inches='tight', dpi=300)

plt.show()


## Mock Data generated with Padè model and analyzed with the correct and wrong ($\epsilon$ const) model

In [ ]:
pade_wrong = { r'$\epsilon_0$': { 
                                r'$\epsilon$ constant Analysis': {'path': './pade_wrong/chains_epsilon0/BAO+SN+GW_MH',
                                                                                'sampler': 'MH',
                                                                                'DDR_model': {'epsilon_model': 'polynomial',
                                                                                        'pade': False,},
                                                                                'Nchains': 4,
                                                                                'plot_pars': ['omegam', 'epsilon0_EM' ],
                                                                                'color': 'forestgreen',
                                                                                'filled': True},
                                'Padè Analysis': {'path': './pade_chains/chains_pade_epsilon0/BAO+SN+GW_MH',
                                                                'sampler': 'MH',
                                                                'DDR_model': {'epsilon_model': 'polynomial',
                                                                        'pade': False,},
                                                                'Nchains': 4,
                                                                'plot_pars': ['omegam', 'epsilon0_EM' ],
                                                                'color': 'orange',
                                                                'filled': True},
                                                
                                        },
                r'$\epsilon_{EM}$ breaking':  { 
                                                r'$\epsilon$ constant Analysis': {'path': './pade_wrong/chains_epsilonEM/BAO+SN+GW_MH',
                                                                'sampler': 'MH',
                                                                'DDR_model': {'epsilon_model': 'polynomial',
                                                                        'pade': False,},
                                                                'Nchains': 4,
                                                                'plot_pars': ['omegam', 'epsilon0_EM' ],
                                                                'color': 'forestgreen',
                                                                'filled': True},
                                                'Padè Analysis': {'path': './pade_chains/chains_pade_epsilonEM/BAO+SN+GW_MH',
                                                                'sampler': 'MH',
                                                                'DDR_model': {'epsilon_model': 'polynomial',
                                                                        'pade': False,},
                                                                'Nchains': 4,
                                                                'plot_pars': ['omegam', 'epsilon0_EM'],
                                                                'color': 'orange',
                                                                'filled': True},
                                },
                    r'$\epsilon$ breaking': {
                                         r'$\epsilon$ constant Analysis': {'path': './pade_wrong/chains_epsilonEMGW/BAO+SN+GW_MH',
                                                                'sampler': 'MH',
                                                                'DDR_model': {'epsilon_model': 'polynomial',
                                                                        'pade': False,},
                                                                'Nchains': 4,
                                                                'plot_pars': ['omegam', 'epsilon0_EM' ],
                                                                'color': 'forestgreen',
                                                                'filled': True},
                                        'Padè Analysis': {'path': './pade_chains/chains_pade_epsilonEMGW/BAO+SN+GW_MH',
                                                                'sampler': 'MH',
                                                                'DDR_model': {'epsilon_model': 'polynomial',
                                                                        'pade': False,},
                                                                'Nchains': 4,
                                                                'color': 'orange',
                                                                'plot_pars': [ 'epsilon0_GW', 'epsilon0_EM'],
                                                                'filled': True},
                                }
}

In [ ]:
analyzed_results_pade_wrong = {model: {name: stats.analyze_chain(name,resdict) for name,resdict in model_dict.items()} 
                         for model,model_dict in pade_wrong.items()}



In [ ]:
plot_pars = [ 'epsilon0_GW', 'epsilon0_EM']
g = plots.get_subplot_plotter(subplot_size=6, width_inch=8, scaling=False)

g.settings.figure_legend_frame = False
g.settings.axes_fontsize = 20
g.settings.axes_labelsize = 20
g.settings.legend_fontsize = 20
g.settings.axis_marker_color = 'black'
g.settings.axis_marker_ls = '--'
g.settings.axis_marker_lw = 1
g.settings.axis_tick_x_rotation = 45


g.plot_2d([result['MCsamples'] for result in analyzed_results_pade_wrong['$\epsilon$ breaking'].values()], 
            plot_pars, 
            plot_pars, 
            filled=[result['filled'] for result in analyzed_results_pade_wrong['$\epsilon$ breaking'].values()], 
            colors=[result['color'] for result in analyzed_results_pade_wrong['$\epsilon$ breaking'].values()], 
            contour_lws=2,
        )


          
        
plt.axvline(
    x=fiducial['$\epsilon$ breaking']['epsilon0_GW'], 
    color='black', 
    linestyle='--', 
    linewidth=1, 
)

plt.axhline(
    y=fiducial['$\epsilon$ breaking']['epsilon0_EM'], 
    color='black', 
    linestyle='--', 
    linewidth=1, 
)
legend = g.add_legend(
    [result for result in analyzed_results_pade_wrong['$\epsilon$ breaking'].keys()],
    legend_loc='upper left',
    frameon=False
)
legend.get_frame().set_linewidth(0)
legend.get_frame().set_edgecolor('none')

g.fig.align_ylabels()
g.fig.align_xlabels()

g.fig.align_ylabels()
g.fig.align_xlabels()

plt.text(0.102, 0.12, r"IVS: $\epsilon_{\rm EM},\epsilon_{\rm GW}$" "\n" r"breaking", color='black', fontsize=20, ha='left')
plt.savefig('epsilonEMGW_padèwrong.png', bbox_inches='tight', dpi=300)

plt.show()


In [ ]:
plot_pars = [ 'omegam', 'epsilon0_EM']
g = plots.get_subplot_plotter(subplot_size=6, width_inch=8, scaling=False)

g.settings.figure_legend_frame = False
g.settings.axes_fontsize = 20
g.settings.axes_labelsize = 20
g.settings.legend_fontsize = 20
g.settings.axis_marker_color = 'black'
g.settings.axis_marker_ls = '--'
g.settings.axis_marker_lw = 1
g.settings.axis_tick_x_rotation = 45


g.plot_2d([result['MCsamples'] for result in analyzed_results_pade_wrong['$\epsilon_{EM}$ breaking'].values()], 
            plot_pars, 
            plot_pars, 
            filled=[result['filled'] for result in analyzed_results_pade_wrong['$\epsilon_{EM}$ breaking'].values()], 
            colors=[result['color'] for result in analyzed_results_pade_wrong['$\epsilon_{EM}$ breaking'].values()], 
            contour_lws=2,
        )


                 
plt.axvline(
    x=fiducial['$\epsilon_{EM}$ breaking']['omegam'], 
    color='black', 
    linestyle='--', 
    linewidth=1, 
)

plt.axhline(
    y=fiducial['$\epsilon_{EM}$ breaking']['epsilon0_EM'], 
    color='black', 
    linestyle='--', 
    linewidth=1, 
)
legend = g.add_legend(
    [result for result in analyzed_results_pade_wrong['$\epsilon_{EM}$ breaking'].keys()],
    legend_loc='best',
    frameon=False
)
legend.get_frame().set_linewidth(0)
legend.get_frame().set_edgecolor('none')

g.fig.align_ylabels()
g.fig.align_xlabels()
g.fig.align_ylabels()
g.fig.align_xlabels()


plt.text(0.305, 0.122, r"IVS: $\epsilon_{\rm EM}$ breaking", color='black', fontsize=20, ha='left')


plt.savefig('epsilonEM_padewrong.png', bbox_inches='tight', dpi=300)
plt.show()




In [ ]:
plot_pars = [ 'omegam', 'epsilon0_EM']
g = plots.get_subplot_plotter(subplot_size=6, width_inch=8, scaling=False)

g.settings.figure_legend_frame = False
g.settings.axes_fontsize = 20
g.settings.axes_labelsize = 20
g.settings.legend_fontsize = 20
g.settings.axis_marker_color = 'black'
g.settings.axis_marker_ls = '--'
g.settings.axis_marker_lw = 1
g.settings.axis_tick_x_rotation = 45


g.plot_2d([result['MCsamples'] for result in analyzed_results_pade_wrong['$\epsilon_0$'].values()], 
            plot_pars, 
            plot_pars, 
            filled=[result['filled'] for result in analyzed_results_pade_wrong['$\epsilon_0$'].values()], 
            colors=[result['color'] for result in analyzed_results_pade_wrong['$\epsilon_0$'].values()], 
            contour_lws=2,
        )
for ax in g.subplots[:, 0]: 
    ax.set_ylabel('') 
          
g.fig.supylabel('$\epsilon_0$') 
        
plt.axvline(
    x=fiducial['$\epsilon_0$']['omegam'], 
    color='black', 
    linestyle='--', 
    linewidth=1, 
)

plt.axhline(
    y=fiducial['$\epsilon_0$']['epsilon0_EM'], 
    color='black', 
    linestyle='--', 
    linewidth=1, 
)
legend = g.add_legend(
    [result for result in analyzed_results_pade_wrong['$\epsilon_0$'].keys()],
    legend_loc='best',
    frameon=False
)
legend.get_frame().set_linewidth(0)
legend.get_frame().set_edgecolor('none')

g.fig.align_ylabels()
g.fig.align_xlabels()
plt.text(0.305, 0.115, r"UVS: $\epsilon_0$ breaking", color='black', fontsize=20, ha='left')


plt.savefig('epsilon0_padewrong.png', bbox_inches='tight', dpi=300)

plt.show()


# Degeneracy

In [ ]:
degeneracy_BAO_SN = { 
                'SN': {'path': './DDR_chains/chains/SN_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                       'color': 'orange',
                                       'filled': True},
                 'BAO': {'path': './DDR_chains/chains/BAO_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                       'color': 'forestgreen',
                                       'filled': True},
    
        }   

In [ ]:
analyzed_results_degeneracy_BAO_SN = {name: stats.analyze_chain(name,resdict) for name,resdict in degeneracy_BAO_SN.items()}
plot_pars = [ 'epsilon0_EM', 'omegam']

In [ ]:
import matplotlib.pyplot as plt
from getdist import plots

g = plots.get_subplot_plotter(subplot_size=1, width_inch=10, scaling=False)

g.settings.figure_legend_frame = False
g.settings.axes_fontsize = 32
g.settings.axes_labelsize = 32
g.settings.legend_fontsize = 32
g.settings.axis_marker_color = 'black'
g.settings.axis_marker_ls = '--'
g.settings.axis_marker_lw = 1
g.settings.axis_tick_x_rotation = 45

g.plot_2d([result['MCsamples'] for result in analyzed_results_degeneracy_BAO_SN.values()], 
            plot_pars, 
            plot_pars, 
            filled=[result['filled'] for result in analyzed_results_degeneracy_BAO_SN.values()], 
            colors=[result['color'] for result in analyzed_results_degeneracy_BAO_SN.values()], 
            contour_lws=2,
        )


g.fig.align_ylabels()
g.fig.align_xlabels()

ax = plt.gca()
ax.set_ylim(0.1, 0.6)
ax.set_xlim(-0.2, 0.2)
g.settings.figure_legend_frame = False


legend = g.add_legend(
    [result for result in analyzed_results_degeneracy_BAO_SN.keys()],
    legend_loc='upper left',
    frameon=False
)
legend.get_frame().set_linewidth(0)
legend.get_frame().set_edgecolor('none')
plt.savefig('BAO+SN.png', bbox_inches='tight', dpi=300)

plt.show()


In [ ]:
degeneracy_BAO_GW = { 
                'GW': {'path': './DDR_chains/chains/GW_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                       'color': 'dodgerblue',
                                       'filled': True},
                'BAO': {'path': './DDR_chains/chains/BAO_MH',
                                       'sampler': 'MH',
                                       'DDR_model': {'epsilon_model': 'polynomial',
                                                     'pade': False,},
                                       'Nchains': 4,
                                       'color': 'forestgreen',
                                       'filled': True},
    
        }   

In [ ]:
analyzed_results_degeneracy_BAO_GW = {name: stats.analyze_chain(name,resdict) for name,resdict in degeneracy_BAO_GW.items()}
plot_pars = ['epsilon0_GW', 'omegam']

In [ ]:
import matplotlib.pyplot as plt
from getdist import plots

g = plots.get_subplot_plotter(subplot_size=1, width_inch=10, scaling=False)

g.settings.figure_legend_frame = False
g.settings.axes_fontsize = 32
g.settings.axes_labelsize = 32
g.settings.legend_fontsize = 32
g.settings.axis_marker_color = 'black'
g.settings.axis_marker_ls = '--'
g.settings.axis_marker_lw = 1
g.settings.axis_tick_x_rotation = 45

g.plot_2d([result['MCsamples'] for result in analyzed_results_degeneracy_BAO_GW.values()], 
            plot_pars, 
            plot_pars, 
            filled=[result['filled'] for result in analyzed_results_degeneracy_BAO_GW.values()], 
            colors=[result['color'] for result in analyzed_results_degeneracy_BAO_GW.values()], 
            contour_lws=2,
        )


g.fig.align_ylabels()
g.fig.align_xlabels()

ax = plt.gca()
ax.set_ylim(0.1, 0.6)
ax.set_xlim(-0.2, 0.2)
g.settings.figure_legend_frame = False

legend = g.add_legend(
    [result for result in analyzed_results_degeneracy_BAO_GW.keys()],
    legend_loc='upper left',
    frameon=False
)
legend.get_frame().set_linewidth(0)
legend.get_frame().set_edgecolor('none')

plt.savefig('BAO+GW.png', bbox_inches='tight', dpi=300)

plt.show()


In [ ]:
fiducial = {'H0': 73.4,
            'omegam': 0.306,
            'ombh2': 0.02218,
            'omk': 0.,
            'epsilon0_EM': 0.1,
            'a_EM': 1.,
            'n_EM': 1.,
            'epsilon0_GW': 0.0,
            'a_GW': 1.,
            'n_GW': 1.,
            'mnu': 0.06,
            'nnu': 3.,
            'MB': -19.2435,
            'rd': 141.99}

settings = {'zmin': 0.001,
            'zmax': 1100.,
            'Nz': 1000,
            'zdrag': 1060,
            'DDR_model': {'epsilon_model': 'polynomial',
                          'pade': False},
            'BBN': True}

epsilon = 0.1

In [ ]:
zcalc = np.linspace(settings['zmin'],settings['zmax'],settings['Nz'])

In [ ]:
from theory_code.DDR_theory import DDRCalcs
from theory_code.distance_theory import TheoryCalcs

In [ ]:
eta_no_pade = DDRCalcs(settings['DDR_model'],fiducial, zcalc).eta_EM(zcalc)
d_L_no_pade = TheoryCalcs(settings,fiducial).DL_EM(zcalc)

In [ ]:
settings['DDR_model']['pade'] = True

In [ ]:
eta_pade = DDRCalcs(settings['DDR_model'],fiducial, zcalc).eta_EM(zcalc)
d_L_pade = TheoryCalcs(settings,fiducial).DL_EM(zcalc)

In [ ]:
plt.figure(figsize=(10,6))


plt.plot(zcalc, eta_no_pade, label='$\epsilon$ constant', color='hotpink', linewidth=3)
plt.plot(zcalc, eta_pade, label='Padè', color='cornflowerblue', linewidth=3)
plt.hlines(y=1, xmin=zcalc.min(), xmax=zcalc.max(), color='grey', linestyle='--', label='$\eta = 1$', linewidth = 2.5)
plt.vlines(x=1100, ymin=1, ymax=2.2, color='grey', linestyle=':')

plt.text(1100, 0.93, r'$ z_*$', ha='center', va='bottom', fontsize=22, color='black')

plt.ylim(0.9, 2.1)
plt.xlabel('$z$', fontsize=26)

ax = plt.gca()
ax.set_ylabel('$\eta(z)$', fontsize=26, labelpad=15)  # labelpad shifts ylabel left
ax.tick_params(axis='both', which='both', labelsize=22, direction='in')
plt.legend(frameon=False, fontsize=22, loc='upper left')
inset_ax = ax.inset_axes([0.5, 0.35, 0.4, 0.35])  # [x0, y0, width, height] in fraction
inset_ax.plot(zcalc, eta_no_pade, color='hotpink')
inset_ax.plot(zcalc, eta_pade, color='cornflowerblue')

inset_ax.set_xlim(0, 3)
inset_ax.set_ylim(1, 1.2)

inset_ax.tick_params(axis='both', which='both', labelsize=14, direction='in')
inset_ax.grid(True, linestyle=':', linewidth=0.5)

# Optional: Add rectangle showing zoomed area on main plot
ax.indicate_inset_zoom(inset_ax, edgecolor="grey", linestyle='--')

plt.tight_layout()

plt.savefig('pade_theo.png', bbox_inches='tight', dpi=300)
plt.show()



In [ ]:
import matplotlib.patches as patches
plt.figure(figsize=(10,6))


plt.plot(zcalc, eta_no_pade, label='$\epsilon$ constant', color='hotpink', linewidth=3)
plt.plot(zcalc, eta_pade, label='Padè', color='cornflowerblue', linewidth=3)
plt.hlines(y=1, xmin=zcalc.min(), xmax=zcalc.max(), color='grey', linestyle=':', label='$\eta = 1$')
plt.vlines(x=1100, ymin=1, ymax=2.2, color='grey', linestyle=':')

plt.text(1100, 0.94, r'$\rm z_{LSS}$', ha='center', va='bottom')

plt.ylim(0.9, 2.1)
plt.xlabel('$z$', fontsize=20)

ax = plt.gca()
ax.set_ylabel('$\eta(z)$', fontsize=26, labelpad=15)  # labelpad shifts ylabel left
ax.tick_params(axis='both', which='both', labelsize=26, direction='in')
plt.legend(frameon=False, fontsize=22, loc='upper left')

# Inset for relative difference
inset_ax = ax.inset_axes([0.5, 0.3, 0.40, 0.45])  # [x0, y0, width, height]
relative_difference = (eta_no_pade - eta_pade) / eta_no_pade

inset_ax.plot(zcalc, relative_difference, color='darkorchid', linewidth=2.5)

inset_ax.set_xlim(1e-3, 1e3)
# inset_ax.set_ylim(1e-9, 0.2)
inset_ax.set_yscale('log')
inset_ax.set_xscale('log')

inset_ax.set_xlabel('$z$', fontsize=20)
inset_ax.set_ylabel('Relative difference', fontsize=14)

inset_ax.tick_params(axis='both', which='both', labelsize=14, direction='in')
inset_ax.grid(True, linestyle=':', linewidth=0.5)

plt.tight_layout()
plt.savefig('pade_theo_with_rel_diff.png', bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
import matplotlib.patches as patches
plt.figure(figsize=(10,6))

relative_difference = (eta_no_pade - eta_pade) / eta_no_pade

plt.plot(zcalc, relative_difference,  color='darkorchid', linewidth=3)

plt.xlabel('$z$', fontsize=28)
plt.xlim(1e-3, 1e3)
plt.xscale('log')
plt.yscale('log')
plt.ylabel(r'$(\eta^{\rm Const}-\eta^{\rm Pade})/\eta^{\rm Const}$', fontsize=26, labelpad=15) 
ax = plt.gca()
plt.legend(frameon=False, fontsize=26)

ax.tick_params(axis='both', which='both', direction='out', labelsize=24)

plt.tight_layout()
plt.savefig('pade_rel_diff.png', bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
plt.figure(figsize=(10,6))
plt.plot(zcalc, d_L_no_pade, label='No Padè', color='hotpink', linewidth=3)
plt.plot(zcalc, d_L_pade, label='Padè', color='cornflowerblue', linewidth=3)
plt.xlim(0,5)
plt.ylim(0,60000 )
plt.xlabel('$z$')
ax = plt.gca()
ax.set_ylabel('$d_L$', fontsize=14, labelpad=15)
plt.legend()
plt.savefig('dL_pade.png', bbox_inches='tight', dpi=300)

plt.show()